In [1]:
!pip install ultralytics

  Using cached torch-2.7.1-cp311-cp311-win_amd64.whl.metadata (28 kB)
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
  Using cached torchvision-0.22.1-cp311-cp311-win_amd64.whl.metadata (6.1 kB)
Using cached torch-2.7.1-cp311-cp311-win_amd64.whl (216.1 MB)
Using cached torchvision-0.22.1-cp311-cp311-win_amd64.whl (1.7 MB)

  Attempting uninstall: torch

    Found existing installation: torch 2.4.0+cu118

   ---------------------------------------- 0/2 [torch]
   ---------------------------------------- 0/2 [torch]
   ---------------------------------------- 0/2 [torch]
    Uninstalling torch-2.4.0+cu118:
   ---------------------------------------- 0/2 [torch]
   ---------------------------------------- 0/2 [torch]
      Successfully uninstalled torch-2.4.0+cu118
   ---------------------------------------- 0/2 [torch]
   ---------------------------------------- 0/2 [torch]
   -----------

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.4.0+cu118 requires torch==2.4.0+cu118, but you have torch 2.7.1 which is incompatible.
xformers 0.0.27.post2+cu118 requires torch==2.4.0, but you have torch 2.7.1 which is incompatible.


In [2]:
!pip uninstall torch torchvision torchaudio -y

Found existing installation: torch 2.7.1
Uninstalling torch-2.7.1:
  Successfully uninstalled torch-2.7.1
Found existing installation: torchvision 0.22.1
Uninstalling torchvision-0.22.1:
  Successfully uninstalled torchvision-0.22.1
Found existing installation: torchaudio 2.4.0+cu118
Uninstalling torchaudio-2.4.0+cu118:
  Successfully uninstalled torchaudio-2.4.0+cu118


In [3]:
!pip install torch==2.4.0 torchvision==0.19.0 torchaudio==2.4.0 --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118
  Using cached https://download.pytorch.org/whl/cu118/torch-2.4.0%2Bcu118-cp311-cp311-win_amd64.whl (2692.5 MB)
  Using cached https://download.pytorch.org/whl/cu118/torchvision-0.19.0%2Bcu118-cp311-cp311-win_amd64.whl (5.0 MB)
  Using cached https://download.pytorch.org/whl/cu118/torchaudio-2.4.0%2Bcu118-cp311-cp311-win_amd64.whl (4.0 MB)

   ---------------------------------------- 0/3 [torch]
   ---------------------------------------- 0/3 [torch]
   ---------------------------------------- 0/3 [torch]
   ---------------------------------------- 0/3 [torch]
   ---------------------------------------- 0/3 [torch]
   ---------------------------------------- 0/3 [torch]
   ---------------------------------------- 0/3 [torch]
   ---------------------------------------- 0/3 [torch]
   ---------------------------------------- 0/3 [torch]
   ---------------------------------------- 0/3 [torch]
   -----------------------------------

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ultralytics 8.3.156 requires torch!=2.4.0,>=1.8.0; sys_platform == "win32", but you have torch 2.4.0+cu118 which is incompatible.


In [4]:
import torch

print(torch.cuda.is_available())

True


In [5]:
!pip install scikit-learn

In [6]:
import os
import shutil
from sklearn.model_selection import train_test_split

from ultralytics import YOLO

WARNING Known issue with torch==2.4.0 on Windows with CPU, recommend upgrading to torch>=2.4.1 to resolve https://github.com/ultralytics/ultralytics/issues/15049


In [7]:
dataset_dir = r"c:\ai_project01\mask_yolo_dataset"

if os.path.exists(dataset_dir):
    shutil.rmtree(dataset_dir)
    print(f"기존 폴더 삭제: {dataset_dir}")

기존 폴더 삭제: c:\ai_project01\mask_yolo_dataset


In [8]:
image_train_dir = os.path.join(dataset_dir, "images/train")
image_val_dir = os.path.join(dataset_dir, "images/val")
label_train_dir = os.path.join(dataset_dir, "labels/train")
label_val_dir = os.path.join(dataset_dir, "labels/val")

In [9]:
for d in [image_train_dir, image_val_dir, label_train_dir, label_val_dir]:
    os.makedirs(d, exist_ok=True)

In [10]:
image_src_dir = r"c:\ai_project01\mask_images"
label_src_dir = r"c:\ai_project01\labels"

In [11]:
image_files = []

for subdir in ['mask_on', 'no_mask']:
    subdir_path = os.path.join(image_src_dir, subdir)

    for f in os.listdir(subdir_path):
        if f.lower().endswith(('jpg', 'png', 'jpeg')):
            full_file_path = os.path.join(subdir_path, f)
            image_files.append(full_file_path)

In [12]:
train_files, val_files = train_test_split(image_files, test_size=0.2, random_state=42)

In [13]:
def move_files(files, image_dest, label_dest):
    
    for img_path in files:
        filename = os.path.basename(img_path)
        label_name = os.path.splitext(filename)[0] + '.txt'
        label_path = os.path.join(label_src_dir, label_name)

        if os.path.exists(label_path):
            shutil.copy2(img_path, image_dest)
            shutil.copy2(label_path, label_dest)
        else:
            print(f"라벨 파일을 찾을 수 없습니다: {img_path}")

In [14]:
move_files(train_files, image_train_dir, label_train_dir)
move_files(val_files, image_val_dir, label_val_dir)

print("데이터 분할 및 파일 이동이 모두 완료되었습니다!")

데이터 분할 및 파일 이동이 모두 완료되었습니다!


In [15]:
data_yaml_path = os.path.join(dataset_dir, "data.yaml")

yaml_content = f"""
train: c:/ai_project01/mask_yolo_dataset/images/train
val: c:/ai_project01/mask_yolo_dataset/images/val

names:
   0: maske_on
   1: no_mask

nc: 2
"""

In [16]:
with open(data_yaml_path, 'w') as file:
    file.write(yaml_content.strip())

print(f"data.yaml 생성 완료: {data_yaml_path}")

data.yaml 생성 완료: c:\ai_project01\mask_yolo_dataset\data.yaml


In [17]:
model = YOLO('yolov8m.pt')

results = model.train(
    data=data_yaml_path,
    epochs=1000,
    imgsz=640,
    batch=16,
    name='mask_detection',
    project='c:/ai_project01/results',
    exist_ok=True,
    patience=30
)

print("\n 학습완료!")

Ultralytics 8.3.156  Python-3.11.11 torch-2.4.0+cu118 CUDA:0 (RTX A4000, 16376MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=c:\ai_project01\mask_yolo_dataset\data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=1000, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8m.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=mask_detection, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=30, perspective=0.0, plots=Tru

train: Scanning C:\ai_project01\mask_yolo_dataset\labels\train... 788 images, 0 backgrounds, 0 corrupt: 100%|██████████


train: New cache created: C:\ai_project01\mask_yolo_dataset\labels\train.cache
val: Fast image access  (ping: 0.10.0 ms, read: 1324.0644.6 MB/s, size: 631.2 KB)


val: Scanning C:\ai_project01\mask_yolo_dataset\labels\val... 197 images, 0 backgrounds, 0 corrupt: 100%|██████████| 19


val: New cache created: C:\ai_project01\mask_yolo_dataset\labels\val.cache
Plotting labels to c:\ai_project01\results\mask_detection\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: SGD(lr=0.01, momentum=0.9) with parameter groups 77 weight(decay=0.0), 84 weight(decay=0.0005), 83 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to c:\ai_project01\results\mask_detection
Starting training for 1000 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

  1000/1000      7.34G          0          0          0          0        640: 100%|██████████| 50/50 [00:14<00:00,  3.
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:01<0

                   all        197        204          0          0          0          0



1000 epochs completed in 4.565 hours.
Optimizer stripped from c:\ai_project01\results\mask_detection\weights\last.pt, 52.1MB
Optimizer stripped from c:\ai_project01\results\mask_detection\weights\best.pt, 52.1MB

Validating c:\ai_project01\results\mask_detection\weights\best.pt...
Ultralytics 8.3.156  Python-3.11.11 torch-2.4.0+cu118 CUDA:0 (RTX A4000, 16376MiB)
Model summary (fused): 92 layers, 25,840,918 parameters, 0 gradients, 78.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:01<0


                   all        197        204          0          0          0          0
              maske_on        101        103          0          0          0          0
               no_mask         96        101          0          0          0          0
Speed: 0.1ms preprocess, 5.6ms inference, 0.0ms loss, 0.4ms postprocess per image
Results saved to c:\ai_project01\results\mask_detection

 학습완료!
